## 0. Importing all necessary libs and tools

In [ ]:
import torch
from ultralytics import YOLO
import os
import time
import sys
from PIL import Image
for module_name in list(sys.modules.keys()):
    if module_name.startswith('src.'):
        del sys.modules[module_name]
from src.main.resources.application import MODEL_NAME, OVERLAPPING_PERCENTAGE, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD
print(MODEL_NAME, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD)
from main.python.tools.model import get_devices, get_models, process_folders, read_annotation_file, create_annotation_file, read_annotation_folder
from src.tools.utils import get_destination_folder, create_destination_folder, get_slice_coordinates, copyfile
from src.tools.image_slicer import create_image_slices
from src.tools.iou_filter import filter_iou
import yaml
import os

config_path = '../src/main/resources/application.yml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
model_name = config['MODEL_NAME']
overlapping_percentage = config['OVERLAPPING_PERCENTAGE']
slice_size = config['SLICE_SIZE']
slices_folder = config['SLICES_FOLDER']
predict_folder_prefix = config['PREDICT_FOLDER_PREFIX']
confidence_threshold = config['CONFIDENCE_THRESHOLD']
outlier_filter_folder_prefix = config['OUTLIER_FILTER_FOLDER_PREFIX']
outlier_threshold_k = config['OUTLIER_THRESHOLD_K']
iou_folder_prefix = config['IOU_FOLDER_PREFIX']
iou_threshold = config['IOU_THRESHOLD']

print(model_name, slice_size, slices_folder, predict_folder_prefix, confidence_threshold, iou_folder_prefix, outlier_filter_folder_prefix, outlier_threshold_k, iou_threshold)


best_4 640 slices predicted_images_with_annotations 0.5 iou_filtered outlier_filtered 3 0.5


## 1. Slicing image process


In [ ]:
images_folder = "dataset/0.0.1/АФС для обработки ИИ"
image_paths = [f"{images_folder}/{image_path}" for image_path in os.listdir(images_folder)]
for image_path in image_paths:
    create_image_slices(
        image_path=image_path, 
        overlap_percentage=OVERLAPPING_PERCENTAGE, 
        destination_folder=SLICES_FOLDER,
        slice_size=SLICE_SIZE
    )

## 1.2 Parallel slicing approach

In [ ]:
from src.tools.image_slicer import create_images_slices_parallel
create_images_slices_parallel()

Processing 211 images using 20 processes


Processing images: 100%|██████████| 211/211 [00:59<00:00,  3.55it/s]


Completed: 211 images, 37136 total slices


## 2. Prediction of saigas on slices 

In [91]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
print(f"Using devices: {devices}")
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD])
output_folder = create_destination_folder(folder_name)
total_detections = process_folders(models[1], devices[1], output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1
Using devices: ['cuda:0', 'cuda:1']
Обработка изображений на устройстве: cuda:1
Папка вывода: predicted_images_with_annotations-best_4-0.5-2

Готово! Обработано 80 изображений с детекциями
Общее количество детекций: 2229
Время обработки: 959.75 секунд
Устройство: cuda:1


## 2.2 Prediction of saigas on slices - parallel

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD])
output_folder = create_destination_folder(folder_name)
total_detections = process_folders_parallel(models, devices, output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1
Обработка 211 папок с детекциями используя 2 процессов


Processing folders:   0%|          | 0/211 [00:00<?, ?it/s]

## 3. Removing duplicates using IOU coefficient  

In [98]:
source_folder = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD])
boxes_list = read_annotation_folder(source_folder)
folder_name = get_destination_folder([IOU_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, IOU_THRESHOLD])
output_folder = create_destination_folder(folder_name)
filtered_boxes_list = filter_iou(boxes_list)
filtered_boxes_list

Всего боксов для сравнения: 2229
intersection_area 753.1552005806006
union_area 753.4952336396091
Боксы с IoU > 0.5:
Бокс 1: [571.717041015625, 462.65380859375, 595.793701171875, 493.9422607421875] (из predicted_images_with_annotations-best_4-0.5/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_000_0_0.png)
Бокс 2: [571.7189254760742, 462.65179443359375, 595.7977066040039, 493.93780517578125] (из predicted_images_with_annotations-best_4-0.5/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_001_512_0.png)
IoU: 0.9995
---
intersection_area 823.5333597064018
union_area 825.2439309265465
Боксы с IoU > 0.5:
Бокс 1: [546.153076171875, 282.1562194824219, 566.991943359375, 321.6753234863281] (из predicted_images_with_annotations-best_4-0.5/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_000_0_0.png)
Бокс 2: [546.1235046386719, 282.147369384765

[{'source_image_path': 'predicted_images_with_annotations-best_4-0.5/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_000_0_0.png',
  'coordinates': [[480.42510986328125,
    408.66192626953125,
    509.46337890625,
    437.17388916015625]]},
 {'source_image_path': 'predicted_images_with_annotations-best_4-0.5/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_001_512_0.png',
  'coordinates': [[633.0119018554688,
    493.16693115234375,
    656.0457153320312,
    523.472412109375],
   [571.7189254760742,
    462.65179443359375,
    595.7977066040039,
    493.93780517578125],
   [730.9208221435547,
    286.1300048828125,
    755.3768157958984,
    335.5069580078125],
   [546.1235046386719,
    282.1473693847656,
    566.9922485351562,
    321.6918640136719],
   [702.5128784179688,
    210.28561401367188,
    719.4403686523438,
    256.3255615234375],
   [584.3135681152344,
    81.52772521

In [99]:
import pandas as pd

df = pd.read_csv('benchmarking/benchmarking.csv')

original_images_folder = "dataset/0.0.1/АФС для обработки ИИ/"
sum_of_expected = 0
sum_of_predicted = 0
for image_path in os.listdir(original_images_folder):
    if image_path.endswith(".JPG"):
        source_image_path = f"{original_images_folder}/{image_path}"
        destination_image_path = f"{output_folder}/{image_path}"
        copyfile(source_image_path, destination_image_path)
        
        with Image.open(source_image_path) as img:
            width, height = img.size
        image_name = os.path.basename(image_path).split('.')[0]
        # print(f"Processing image: {image_name}")
        matching_boxes = [box for box_group in filtered_boxes_list
                         if image_name in box_group['source_image_path']
                         for box in box_group['coordinates']]
        row_index = df[df["image_name"] == image_name].index[0]
        expected_count = df.at[row_index, "number_of_saigas"]
        predicted_count = len(matching_boxes)
        df.at[row_index, "predicted_count"] = int(predicted_count)
        sum_of_expected += expected_count
        sum_of_predicted += predicted_count
        print(f"Image: {image_name}, Expected: {expected_count}, Actual: {predicted_count}")

        # print(f"Image: {image_path}, Boxes: {len(matching_boxes)}")
        create_annotation_file(
            image_path=destination_image_path,
            boxes=matching_boxes,
            classes_folder=output_folder,
            image_size=(width, height)
        )
cummulative_accuracy = 1 - abs(sum_of_expected - sum_of_predicted) / sum_of_expected if sum_of_expected != 0 else (1 if sum_of_predicted == 0 else 0)
print(f"Cummulative Accuracy: {cummulative_accuracy:.2f}")
df.to_csv('benchmarking/benchmarking_with_predictions.csv', index=False)

Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_090, Expected: 70, Actual: 64
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_091, Expected: 271, Actual: 226
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_092, Expected: 42, Actual: 37
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_093, Expected: 62, Actual: 61
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_094, Expected: 72, Actual: 77
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_095, Expected: 63, Actual: 69
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_096, Expected: 0, Actual: 3
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_097, Expected: 11, Actual: 12
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_098, Expected: 7, Actual: 6
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_099, Expected: 0, Actual: 0
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_100, Expected: 0, Actual: 3
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_101, Expected: 34, Actual: 36
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_102, Expected: 0, Actu

In [93]:
import pandas as pd
import numpy as np

df = pd.read_csv('benchmarking/benchmarking.csv')
original_images_folder = "dataset/0.0.1/АФС для обработки ИИ/"

for image_path in os.listdir(original_images_folder):
    if image_path.endswith(".JPG"):
        source_image_path = f"{original_images_folder}/{image_path}"
        destination_image_path = f"{output_folder}/{image_path}"
        copyfile(source_image_path, destination_image_path)
       
        with Image.open(source_image_path) as img:
            width, height = img.size
        image_name = os.path.basename(image_path).split('.')[0]
        
        matching_boxes = [box for box_group in filtered_boxes_list
                         if image_name in box_group['source_image_path']
                         for box in box_group['coordinates']]
        row_index = df[df["image_name"] == image_name].index[0]
        expected_count = df.at[row_index, "number_of_saigas"]
        predicted_count = len(matching_boxes)
        df.at[row_index, "predicted_count"] = int(predicted_count)
        
        print(f"Image: {image_name}, Expected: {expected_count}, Predicted: {predicted_count}")
        
        create_annotation_file(
            image_path=destination_image_path,
            boxes=matching_boxes,
            classes_folder=output_folder,
            image_size=(width, height)
        )

# Calculate proper metrics with zero-handling
df['absolute_error'] = abs(df['number_of_saigas'] - df['predicted_count'])

# Method 1: Exclude images with 0 expected count from percentage metrics
df_non_zero = df[df['number_of_saigas'] > 0].copy()
df_non_zero['percentage_error'] = (df_non_zero['absolute_error'] / df_non_zero['number_of_saigas']) * 100
df_non_zero['per_image_accuracy'] = 1 - (df_non_zero['absolute_error'] / df_non_zero['number_of_saigas'])

# Calculate metrics on all images
mae_all = df['absolute_error'].mean()
rmse_all = np.sqrt((df['absolute_error'] ** 2).mean())

# Calculate metrics on non-zero images only
mae_non_zero = df_non_zero['absolute_error'].mean()
mape = df_non_zero['percentage_error'].mean()
mean_accuracy = df_non_zero['per_image_accuracy'].mean()
median_accuracy = df_non_zero['per_image_accuracy'].median()

# Additional useful metrics
total_expected = df['number_of_saigas'].sum()
total_predicted = df['predicted_count'].sum()
num_images_with_zero = (df['number_of_saigas'] == 0).sum()
num_false_positives_on_zero = ((df['number_of_saigas'] == 0) & (df['predicted_count'] > 0)).sum()
num_images_total = len(df)

print("\n=== Accuracy Metrics (All Images) ===")
print(f"Total Images: {num_images_total}")
print(f"Images with 0 expected: {num_images_with_zero}")
print(f"False positives on empty images: {num_false_positives_on_zero}")
print(f"Mean Absolute Error (MAE): {mae_all:.2f} objects per image")
print(f"Root Mean Square Error (RMSE): {rmse_all:.2f} objects per image")

print(f"\n=== Accuracy Metrics (Non-Zero Images Only) ===")
print(f"Images analyzed: {len(df_non_zero)}")
print(f"Mean Absolute Error (MAE): {mae_non_zero:.2f} objects per image")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Mean Per-Image Accuracy: {mean_accuracy:.2%}")
print(f"Median Per-Image Accuracy: {median_accuracy:.2%}")

print(f"\n=== Total Counts ===")
print(f"Total Expected: {total_expected}")
print(f"Total Predicted: {total_predicted}")
print(f"Total Difference: {total_predicted - total_expected} ({((total_predicted - total_expected) / total_expected * 100):.1f}%)")
print(f"Relative Count Accuracy: {(1 - abs(total_predicted - total_expected) / total_expected):.2%}")


# Save detailed results
df.to_csv('benchmarking/benchmarking_with_predictions.csv', index=False)

Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_090, Expected: 70, Predicted: 68
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_091, Expected: 271, Predicted: 228
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_092, Expected: 42, Predicted: 38
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_093, Expected: 62, Predicted: 62
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_094, Expected: 72, Predicted: 77
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_095, Expected: 63, Predicted: 69
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_096, Expected: 0, Predicted: 4
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_097, Expected: 11, Predicted: 12
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_098, Expected: 7, Predicted: 7
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_099, Expected: 0, Predicted: 0
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_100, Expected: 0, Predicted: 3
Image: 2025_08_05_PhotoRieboR4_g201b201078_f003_101, Expected: 34, Predicted: 36
Image: 2025_08_05_PhotoRieboR4_g20

In [ ]:
predicted_images_path = "predicted_images_with_annotations-best_4-0.4-2"
count = 0
for folder in os.listdir(predicted_images_path):
    folder_path = f"{predicted_images_path}/{folder}"
    for file in os.listdir(folder_path):
        if file.endswith(".txt") and not file.startswith("classes"):
            file_path = f"{folder_path}/{file}"
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            print(f"File: {file_path}, Boxes: {len(lines)}")
            count += len(lines)
print(f"Total boxes: {count}")

In [94]:
from sklearn.metrics import r2_score
r2 = r2_score(y_true=df['number_of_saigas'], y_pred=df['predicted_count'])

In [95]:
r2

0.9211000669576944